# Real QCNN Training

In [1]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

2023-02-21 12:31:38.673164: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from qiskit import IBMQ

# KQC Token
TOKEN = "b5b31be2bb5237d52e8bdef143913cd62425eadcbda5c56c6f8ed8d067573ac0e07aa41d23d0bfcc880882a4142e2f812dad876fb3208d5c5f8f321be6626cce"
IBMQ.save_account(TOKEN, overwrite=True)

IBMQ.load_account() # Load account from disk
provider = IBMQ.get_provider(hub='ibm-q-kqc', group='internal', project='reservations')
#provider = IBMQ.get_provider(hub='ibm-q-kqc', group='kqc-research', project='trainable-kernel')


backend = "ibm_lagos"

dev = qml.device(
    'qiskit.ibmq',
    wires=4,
    shots=1024,
    backend=backend,
    provider=provider
)


PATH_Model2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model2.pt'
PATH_Model3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model3.pt'


/var/folders/g3/xz_ycw3d0qdd8s4r2rdd6m940000gn/T/ipykernel_7375/3078564450.py:5: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.save_account(TOKEN, overwrite=True)
/var/folders/g3/xz_ycw3d0qdd8s4r2rdd6m940000gn/T/ipykernel_7375/3078564450.py:5: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.save_account(TOKEN, overwrite=True)


# Real QCNN

Hyperparameters

In [16]:
steps = 1
learning_rate = 0.1
batch_size = 1
ansatz = 'TTN'

QCNN

In [17]:
class x_transform2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 7)
        )
        
    def forward(self, x):
        x = self.linear_relu_stack2(x)
        return x.detach().numpy()


class x_transform3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 7
        self.fc = torch.nn.Linear(7 * 7, 7, bias=True)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 7 * 7)
        x = self.fc(x)
        return x.detach().numpy()

model2 = x_transform2()
model3 = x_transform3()

In [18]:
def statepreparation(x, Trained):
    if Trained == False:
        embedding.Noisy_Four_QuantumEmbedding1(x)
    elif Trained == 'Model2':
        model2.load_state_dict(torch.load(PATH_Model2))
        x = model2(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x)
    elif Trained == 'Model3':
        model3.load_state_dict(torch.load(PATH_Model3))
        x = model3(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x[0])
    

@qml.qnode(dev, diff_method="parameter-shift")
def Noisy_QCNN_classifier(params, x, Trained):
    param1 = params[0:2]
    param2 = params[2:4]

    statepreparation(x, Trained)
    embedding.U_TTN(param1, wires=[0,1])
    embedding.U_TTN(param1, wires=[3,2])
    embedding.U_TTN(param2, wires=[1,2])
    return qml.expval(qml.PauliZ(2))


def Linear_Loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss += 0.5 * (1 - l * p)
    return loss / len(labels)


def cost(weights, X_batch, Y_batch, Trained):
    preds = [Noisy_QCNN_classifier(weights, x, Trained) for x in X_batch]
    return Linear_Loss(Y_batch, preds)


def circuit_training(X_train, Y_train, Trained):

    weights = np.random.random(4, requires_grad = True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history = []
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        weights, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, Trained),
                                                     weights)
        loss_history.append(cost_new)
        print("iteration: ", it, " cost: ", cost_new)
    return loss_history, weights

# Model 2

In [14]:

feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [8]:

Loss_histories_not_trained, weights_not_trained, Loss_histories_Model2, weights_Model2 = [], [], [], []

for i in range(1):
    loss_not_trained, weight_not_trained = circuit_training(X_train, Y_train, False)
    loss_Model2, weight_Model2 = circuit_training(X_train, Y_train, 'Model2')

    Loss_histories_not_trained.append(loss_not_trained)
    Loss_histories_Model2.append(loss_Model2)

    weights_not_trained.append(weight_not_trained)
    weights_Model2.append(weight_Model2)

Loss_histories_not_traiend, Loss_histories_Model2 =  np.array(Loss_histories_not_trained), np.array(Loss_histories_Model2)


f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Loss_not_trained.txt', 'a')
f.write(str(Loss_histories_not_trained))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/weights_not_trained.txt', 'a')
f.write(str(weights_not_trained))
f.close()


f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Loss_Model2.txt', 'a')
f.write(str(Loss_histories_Model2))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/weights_Model2.txt', 'a')
f.write(str(weights_Model2))
f.close()

/usr/local/anaconda3/envs/QC/lib/python3.10/site-packages/pennylane/math/utils.py:223: UserWarning: Contains tensors of types {'autograd', 'torch'}; dispatch will prioritize TensorFlow, PyTorch, and  Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(
/usr/local/anaconda3/envs/QC/lib/python3.10/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


iteration:  0  cost:  0.5533854166666666
iteration:  1  cost:  0.5325520833333334
iteration:  2  cost:  0.4248046875
iteration:  3  cost:  0.5595703125
iteration:  4  cost:  0.5166015625
iteration:  5  cost:  0.5045572916666666
iteration:  6  cost:  0.4208984375
iteration:  7  cost:  0.5944010416666666
iteration:  8  cost:  0.4671223958333333
iteration:  9  cost:  0.5559895833333334


/var/folders/g3/xz_ycw3d0qdd8s4r2rdd6m940000gn/T/ipykernel_7375/3053866874.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = model2(torch.tensor(x))


iteration:  0  cost:  0.4876302083333333
iteration:  1  cost:  0.4527994791666667
iteration:  2  cost:  0.4912109375
iteration:  3  cost:  0.443359375
iteration:  4  cost:  0.44921875
iteration:  5  cost:  0.5660807291666666
iteration:  6  cost:  0.5693359375
iteration:  7  cost:  0.4619140625
iteration:  8  cost:  0.4661458333333333
iteration:  9  cost:  0.4853515625


In [19]:
loss_Model2, weight_Model2 = circuit_training(X_train, Y_train, 'Model2')

/var/folders/g3/xz_ycw3d0qdd8s4r2rdd6m940000gn/T/ipykernel_7375/3615019608.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = model2(torch.tensor(x))


iteration:  0  cost:  0.4658203125


# Model3

## 0. Getting Started

In [ ]:
feature_reduction = False
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

In [ ]:
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
X_train, X_test = X_train.permute(0, 3, 1, 2), X_test.permute(0, 3, 1, 2)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
Loss_histories_Model3, weights_Model3 = [], []
for i in range(3):

    loss_Model3, weight_Model3 = circuit_training(X_train, Y_train, 'Model3')

    Loss_histories_Model3.append(loss_Model3)
    weights_Model3.append(weight_Model3)

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Loss_Model3.txt', 'a')
f.write(str(Loss_histories_Model3))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/weights_Model3.txt', 'a')
f.write(str(weights_Model3))
f.close()